In [1]:
import pandas as pd
data_fold = "/sbgenomics/project-files/"
file = "vaccine_status.csv"
a = pd.read_csv(data_fold + file)
# b=a.loc[a['PARTICIPANT_ID']=="RA12016"]

/tmp/ipykernel_7115/1079838051.py:4: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  a = pd.read_csv(data_fold + file)


In [74]:
import os

# files = os.listdir(data_fold)
# for i in range(len(files)):
#     print(i,files[i])

In [ ]:
# b.head()
# f="/Recover2507/project-files/RECOVERAdult_BiostatsDerived_202412_symptoms_deID.csv"
path = "/Recover2507/project-files/"
# file = 'RECOVERAdult_BDC_202412_answerdata_deID.tsv'
df0 = pd.read_csv(data_fold+files[0], sep='\t', encoding='latin1')
# treat = df0[df0['FORM_NAME']=='recent_covid_treatment']


In [10]:
import pandas as pd
import numpy as np

def optimized_reorganize(df):
    # 1. Clean strings once
    df['FIELD_NAME'] = df['FIELD_NAME'].str.strip()
    df['VISIT_TYPE'] = df['VISIT_TYPE'].str.strip()
    
    # --- PART A: VACCINE DATA (v_1, v_2...) ---
    # Extract the numeric index from field names like 'vacc_vaccdt_1'
    # This creates a temporary column 'idx_num'
    v_mask = df['FIELD_NAME'].str.contains('vacc_vaccdt_|vacc_vacctype_', na=False)
    v_df = df[v_mask].copy()
    v_df['idx_num'] = v_df['FIELD_NAME'].str.extract(r'(\d+)').astype(int)
    
    # Separate dates (vaccdt) and names (vacctype)
    v_dates = v_df[v_df['FIELD_NAME'].str.contains('vaccdt')].rename(columns={'ANSWER_TEXT_VAL': 'date'})
    v_names = v_df[v_df['FIELD_NAME'].str.contains('vacctype')].rename(columns={'ANSWER_LABEL': 'name'})
    
    # Merge dates and names on ID and the extracted index
    df_vaccine = pd.merge(
        v_dates[['PARTICIPANT_ID', 'idx_num', 'date']], 
        v_names[['PARTICIPANT_ID', 'idx_num', 'name']], 
        on=['PARTICIPANT_ID', 'idx_num']
    ).rename(columns={'idx_num': 'index', 'PARTICIPANT_ID': 'id'})

    # --- PART B: ENROLLMENT ---
    enroll_mask = df['VISIT_TYPE'] == 'enrollment'
    df_enroll = df[enroll_mask].groupby('PARTICIPANT_ID').first().reset_index()
    df_enroll = pd.DataFrame({
        'id': df_enroll['PARTICIPANT_ID'],
        'index': 0,
        'date': df_enroll['DATA_ENTRY_DATE'],
        'name': 'enrollment'
    })

    # --- PART C: FOLLOWUPS ---
    fu_mask = (df['VISIT_TYPE'].str.contains('followup_', na=False)) & \
              (df['FIELD_NAME'] == 'vacc_vaccyn_fu')
    df_fu = df[fu_mask].copy()
    df_fu = pd.DataFrame({
        'id': df_fu['PARTICIPANT_ID'],
        'index': df_fu['VISIT_TYPE'], # Keeps 'followup_1', 'followup_2'
        'date': df_fu['DATA_ENTRY_DATE'],
        'name': df_fu['ANSWER_LABEL']
    })
    final_df = pd.concat([df_enroll, df_vaccine, df_fu], ignore_index=True)
    
    # Remove rows that are exactly the same across all columns
    final_df = final_df.drop_duplicates().reset_index(drop=True)
    
    # Optional: Sort by ID and Index to make it readable
    # Note: 'index' contains both ints and strings, so sorting might need care
    return final_df
    # --- FINAL COMBINE ---
    return pd.concat([df_enroll, df_vaccine, df_fu], ignore_index=True)

# Process the whole dataframe at once (No participant loop needed!)
merged_df = optimized_reorganize(a)
print(merged_df)

              id       index        date        name
0           RA11           0  2022-01-01  enrollment
1          RA110           0  2023-01-18  enrollment
2         RA1100           0  2022-01-01  enrollment
3        RA11000           0  2022-01-03  enrollment
4       RA110000           0  2022-01-01  enrollment
...          ...         ...         ...         ...
164655  RA110467  followup_2  2022-08-22  Don't know
164656   RA18351  followup_2  2022-07-04  Don't know
164657  RA113868  followup_3  2022-10-21  Don't know
164658   RA14321  followup_3  2022-10-02  Don't know
164659   RA16255  followup_3  2023-09-16  Don't know

[164660 rows x 4 columns]


In [14]:
# [164660 rows x 4 columns]
print(len(np.unique(merged_df['id']))) # 
merged_df[merged_df['id']=='RA11']
merged_df.to_csv('')

14499


,id,index,date,name
0,RA11,0,2022-01-01,enrollment
35451,RA11,1,2020-07-13,Pfizer
53296,RA11,2,2020-08-03,Pfizer
65605,RA11,3,2021-03-27,Pfizer
72662,RA11,4,2022-03-21,Pfizer
77934,RA11,followup_2,2022-07-06,No
83403,RA11,followup_3,2022-09-15,No
89090,RA11,followup_4,2022-12-17,No
94774,RA11,followup_5,2023-03-27,No
100543,RA11,followup_6,2023-06-11,No


In [9]:
import pandas as pd

def reorganize_data(df):
    results = []
    i = 1
    
    while True:
        # Construct the keys (handling potential spaces like "v_1 ")
        v_key = f"vacc_vaccdt_{i}"
        p_key = f"vacc_vacctype_{i}"
        
        # Filter rows where Column A matches the keys (ignoring extra whitespace)
        v_row = df[df['DATA_FIELD_NAME'].str.strip() == v_key]
        p_row = df[df['FIELD_NAME'].str.strip() == p_key]
        
        # Stop condition: If we can't find the next v_i or p_i, terminate the loop
        if v_row.empty or p_row.empty:
            break
            
        # Get the values from Column B
        # v_val will serve as both 'date' and 'content' as per the revised requirement
        v_val = v_row.iloc[0]['ANSWER_TEXT_VAL']
        p_val = p_row.iloc[0]['ANSWER_LABEL']
        # print("v_row",p_row)
        # Append to the list
        results.append({
            'date': v_val,      # Column B value of v_i
            'index': i,   # Column B value of v_i
            'name': p_val      # Column B value of p_i
        })
        
        i += 1
    results.append({
            'date': df[df['VISIT_TYPE']=='enrollment'].iloc[0]['DATA_ENTRY_DATE'],      # Column B value of v_i
            'index': '0',   # Column B value of v_i
            'name': "enrollment"      # Column B value of p_i
        })
    pdresult = pd.DataFrame(results)
    pdresult['id'] = df.iloc[0]['PARTICIPANT_ID']
    
    return pdresult

def follwup_data(df):
    results = []
    i = 1
    
    while True:
        # Construct the keys (handling potential spaces like "v_1 ")
        v_key = f"followup_{i}"
        p_key = f"vacc_vaccyn_fu"
        # Filter rows where Column A matches the keys (ignoring extra whitespace)
        v_row = df[df['VISIT_TYPE'].str.strip() == v_key]
        p_row = v_row[v_row['FIELD_NAME'].str.strip() == p_key]
        
        # Stop condition: If we can't find the next v_i or p_i, terminate the loop
        if v_row.empty or p_row.empty:
            break
            
        # Get the values from Column B
        # v_val will serve as both 'date' and 'content' as per the revised requirement
        v_val = v_row.iloc[0]['DATA_ENTRY_DATE']
        p_val = p_row.iloc[0]['ANSWER_LABEL']
        # print("v_row",p_row)
        # Append to the list
        results.append({
            'date': v_val,      # Column B value of v_i
            'index': v_key,   # Column B value of v_i
            'name': p_val      # Column B value of p_i
        })
        
        i += 1
    pdresult = pd.DataFrame(results)
    pdresult['id'] = df.iloc[0]['PARTICIPANT_ID']
    return pdresult


# Example usage:
# df = pd.read_csv('your_file.csv')
ids = np.unique(a['PARTICIPANT_ID'].values)

for each in ids:
    df = a[a['PARTICIPANT_ID']==each]
    merged_df = reorganize_data(a)
    df2 = follwup_data(a)
    merged_df = pd.concat([merged_df, df2], ignore_index=True, axis=0)
    
# received one or more additional COVID vaccine
# shots since you last answered the survey on
print(merged_df.head(5))

KeyboardInterrupt: 

In [3]:
a.columns

Index(['PARTICIPANT_ID', 'VISIT_ID', 'VISIT_TYPE', 'DATA_ENTRY_DATE',
       'FORM_NAME', 'INSTANCE_NUM', 'FIELD_NAME', 'DATA_FIELD_NAME',
       'FIELD_TYPE', 'ANSWER_LABEL', 'ANSWER_NUMERIC_VAL', 'ANSWER_TEXT_VAL',
       'CONECPT_CD', 'CONCEPT_NAME'],
      dtype='object')

In [5]:
import numpy as np
uid = np.unique(a['PARTICIPANT_ID'])

In [7]:
len(uid)#14,499
a.shape #342,179

(342179, 14)

In [16]:
b=a.loc[a['PARTICIPANT_ID']=="RA12016"]

In [17]:
np.unique(b['VISIT_TYPE'])

array(['enrollment', 'followup_1', 'followup_2'], dtype=object)

In [41]:
b.iloc[10]

PARTICIPANT_ID                                                  RA12016
VISIT_ID                                               followup_2_arm_1
VISIT_TYPE                                                   followup_2
DATA_ENTRY_DATE                                              2023-05-25
FORM_NAME                                                vaccine_status
INSTANCE_NUM                                                          1
FIELD_NAME                                                  vacc_colldt
DATA_FIELD_NAME                                             vacc_colldt
FIELD_TYPE                                                         text
ANSWER_LABEL                                                        NaN
ANSWER_NUMERIC_VAL                                                  NaN
ANSWER_TEXT_VAL                                              2023-05-25
CONECPT_CD                                   RC-RA1:vacc_colldt|Inf|v18
CONCEPT_NAME          Date Vaccination Questions Completed [Infe

In [67]:
# for x in b.loc[b['VISIT_TYPE']=='followup_2']#['CONCEPT_NAME']:
    # print(x)
# b.loc[b['VISIT_TYPE']=='followup_1']['VISIT_TYPE']



8249      followup_1
158686    followup_1
193996    followup_1
284675    followup_1
Name: VISIT_TYPE, dtype: object

In [31]:
# b.groupby('VISIT_TYPE')